In [13]:
import pandas as pd
dataset=pd.read_csv("train_set.tsv", sep='\t', header=0)

x = dataset.iloc[:,:-2]
y = dataset.iloc[:,-2:]
num_collisions=dataset.iloc[:,-2]
min_CPA=dataset.iloc[:,-1]


In [28]:
x.iloc[1]

UAV_1_track           4.023101
UAV_1_x          -17220.612570
UAV_1_y           47439.586909
UAV_1_vx           -167.653031
UAV_1_vy           -138.169553
UAV_1_target_x   -98139.258988
UAV_1_target_y   -19248.698541
UAV_2_track           2.614451
UAV_2_x           33573.808959
UAV_2_y           51374.559396
UAV_2_vx            122.667428
UAV_2_vy           -210.738251
UAV_2_target_x    50087.298775
UAV_2_target_y    23004.974807
UAV_3_track           4.069773
UAV_3_x          -65763.907792
UAV_3_y            9374.271112
UAV_3_vx           -165.924785
UAV_3_vy           -124.214263
UAV_3_target_x   -94877.932557
UAV_3_target_y   -12421.008758
UAV_4_track           3.010504
UAV_4_x           50062.068903
UAV_4_y            -878.418113
UAV_4_vx             33.541980
UAV_4_vy           -254.404983
UAV_4_target_x    54579.884869
UAV_4_target_y   -35144.579226
UAV_5_track           4.315806
UAV_5_x           13285.311651
UAV_5_y          -41245.545752
UAV_5_vx           -196.978758
UAV_5_vy

In [17]:
from sklearn import tree
model=tree.DecisionTreeClassifier()
model.fit(x,num_collisions)


DecisionTreeClassifier()

In [40]:
print((model.predict(x)-num_collisions).sum())


0
